# Notebook that generates my README

In [ ]:
import json

from collections import Counter
from datetime import datetime
import jinja2
import matplotlib.pyplot as plt
import matplotlib
import generate_data as gd

plt.xkcd()
matplotlib.rcParams.update({"font.family": ["Humor Sans"]})  # avoids findfont warnings

In [ ]:
_ = await gd.generate_data()

In [ ]:
with open("data/repos.json", "r") as f:
    repos = json.load(f)

with open("data/most_stars.json", "r") as f:
    most_stars = json.load(f)

with open("data/most_committed.json", "r") as f:
    most_committed = json.load(f)

with open("data/stargazers.json", "r") as f:
    stargazers = json.load(f)

with open("data/all_commit_dates.json", "r") as f:
    all_commit_dates = [datetime.fromisoformat(date) for date in json.load(f)]

with open("data/day_hist.json", "r") as f:
    day_hist = json.load(f)

with open("data/hour_hist.json", "r") as f:
    hour_hist = json.load(f)

## Number of stars ⭐️

In [ ]:
most_stars_strs = [
    f"{i+1}. [{repo['full_name']}](https://github.com/{repo['full_name']}/), {repo['stargazers_count']} ⭐️s"
    for i, repo in enumerate(most_stars[:20])
]

most_stars_strs_summary, most_stars_strs_rest = gd.split(most_stars_strs, 5)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.set_xlabel("Year")
ax.set_ylabel("Cumulative number of stars")
ax.set_title("Total number of stars over time")

for r, stars in zip(most_stars, stargazers):
    n_stars = list(range(1, len(stars) + 1))
    ax.plot(stars, n_stars, label=r["full_name"])
plt.savefig("stars_over_time.png")

In [ ]:
dts = sorted(sum(stargazers, []))
n_stars = list(range(1, len(dts) + 1))

fig, ax = plt.subplots(figsize=(7, 5))
n_stars = list(range(1, len(dts) + 1))
ax.plot(dts, n_stars)
ax.set_xlabel("Year")
ax.set_ylabel("Cumulative number of stars")
ax.set_title("Total number of stars over time")
plt.savefig("stars_over_time.png")

## Number of commits 

In [ ]:
most_committed_strs = [
    f"{i+1}. [{full_name}](https://github.com/{full_name}/), {n_commits} commits :octocat:"
    for i, (full_name, n_commits) in enumerate(most_committed[:20])
]

most_committed_strs_summary, most_committed_strs_rest = split(most_committed_strs, 5)

## Commit stats

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))
ax.bar(*zip(*day_hist))
ax.set_xlabel("Day of the week")
ax.set_ylabel("Total number of commits")
ax.set_title("Commits by day")
plt.savefig("commits_per_weekday.png")

In [ ]:
hour_hist = [
    (f"{i:02d}", n)
    for i, n in sorted(Counter([d.hour for d in all_commit_dates]).items())
]

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))
ax.bar(*zip(*hour_hist))
ax.set_xlabel("Hour of the day")
ax.set_ylabel("number of commits")
ax.set_title("Commits by hour")
plt.savefig("commits_per_hour.png")

## Render template

In [ ]:
template = """### Welcome to my profile 👋

<center>
  <table>
    <tr>
        <td><img width="300px" align="left" src="https://github-readme-stats.vercel.app/api/top-langs/?username=basnijholt&hide=TeX,Jupyter%20Notebook&layout=compact&theme=radical" /></td>
        <td><img align='right' src="https://github-readme-stats.vercel.app/api?username=basnijholt&show_icons=true&theme=radical" width="380"></td>
    </tr>
  </table>
</center>

![visitors](https://visitor-badge.glitch.me/badge?page_id=basnijholt.visitor-badge)

I am Bas. Here I present some (automatically generated) statistics about my activity on GitHub. For more info check out my website [www.nijho.lt](http://nijho.lt/).

![](https://www.nijho.lt/authors/admin/avatar_hu9e60e4b9bc120dfb6a666009f2878da6_182107_250x250_fill_q90_lanczos_center.jpg)

- 💬 Ask me about Python, home-automation, landscape photography, and quantum physics
- 📫 How to reach me: bas@nijho.lt

Last updated at {{ now }}.

# GitHub statistics — my top 20

## number of GitHub stars ⭐️

{{ "\n".join(most_stars_strs_summary) }}
<details><summary>Click to expand!</summary>

{{ "\n".join(most_stars_strs_rest) }}

</details>

![](https://github.com/basnijholt/basnijholt/raw/main/stars_over_time.png)

## number of commits :octocat:

{{ "\n".join(most_committed_strs_summary) }}
<details><summary>Click to expand!</summary>

{{ "\n".join(most_committed_strs_rest) }}

</details>

![](https://github.com/basnijholt/basnijholt/raw/main/commits_per_hour.png)

![](https://github.com/basnijholt/basnijholt/raw/main/commits_per_weekday.png)


"""
txt = jinja2.Template(template).render(
    most_stars_strs_summary=most_stars_strs_summary,
    most_stars_strs_rest=most_stars_strs_rest,
    most_committed_strs_summary=most_committed_strs_summary,
    most_committed_strs_rest=most_committed_strs_rest,
    now=str(datetime.now()),
)
with open("README.md", "w") as f:
    f.write(txt)
print(txt)